In [3]:
%cd ..

/tf/notebooks


In [4]:
import json
import re

In [5]:
with open("data/recipes/amounts.json", encoding="utf-8") as f:
    amounts = json.load(f)

In [6]:
re1 = re.compile(r"^([\d½¼¾⅛]+(?: [½¼¾⅛])?(?:,\d+)?)(?: ((?:kl\. |gr\. )?\w+(?:, gestr\.|, gehäuft)?)(?:/n|/e|\(n\)|\(e\)|\.)?)?$")
re_singol = re.compile(r"^(kl. |gr. )?(Tasse|mg|ml|Tüte|Kopf|Stück|Spritzer|Teil|Becher|Blatt|Zehe|Blätter|Bund|Dose|[ET]L(,.*)?|cl|dl|Flasche|Fläschchen|Glas|Gläser|Handvoll|Körner|Liter|Msp.|Paar|Paket|Pck.|Pkt.|Port.|Prise.*|Scheibe)(/[en])?$")


In [7]:
replacings = "gestr.=gestrichen,geh.=gehäuft,m.-große=mittelgroße,m.-großer=mittelgroßer,TL=Teelöffel,Eßlöffel=Esslöffel,EL=Esslöffel,Msp.=Messerspitze,Pck.=Packung,Pkt.=Packung,Port.=Portion".split(",")

replacings = {ee.split("=")[0]: ee.split("=")[1] for ee in replacings}

In [17]:
parsed1 = []
for _amount in amounts:
    amount = _amount
    if amount == "":
        amount = "1"
    #amount = amount.replace("m.-große", "mittelgroße")
    if re_singol.match(amount):
        amount = "1 " + amount
    for bef, aft in replacings.items():
        amount = re.sub(r"\b" + re.escape(bef) + r"(\b|$)", aft, amount)
    m = re1.match(amount)
    if m is not None:
        parsed1.append((_amount, *m.groups()))
    else:
        print("UNMATCHED  ", _amount, amount)
        parsed1.append((_amount, None, None))

UNMATCHED   0,2 TL, gestr. 0,2 Teelöffel, gestrichen
UNMATCHED   0,3 TL, gestr. 0,3 Teelöffel, gestrichen
UNMATCHED   0,33 EL, gestr. 0,33 Esslöffel, gestrichen
UNMATCHED   0,33 TL, gestr. 0,33 Teelöffel, gestrichen
UNMATCHED   1 EL, gestr. 1 Esslöffel, gestrichen
UNMATCHED   1 TL, gestr. 1 Teelöffel, gestrichen
UNMATCHED   1 einige Stiele 1 einige Stiele
UNMATCHED   1 m.-großes 1 m.-großes
UNMATCHED   1 n. B. 1 n. B.
UNMATCHED   1 ½ EL, gestr. 1 ½ Esslöffel, gestrichen
UNMATCHED   1 ½ TL, gestr. 1 ½ Teelöffel, gestrichen
UNMATCHED   10 EL, gestr. 10 Esslöffel, gestrichen
UNMATCHED   10 TL, gestr. 10 Teelöffel, gestrichen
UNMATCHED   10 einige Stiele 10 einige Stiele
UNMATCHED   10 n. B. 10 n. B.
UNMATCHED   12 TL, gestr. 12 Teelöffel, gestrichen
UNMATCHED   125 n. B. 125 n. B.
UNMATCHED   14 TL, gestr. 14 Teelöffel, gestrichen
UNMATCHED   15 EL, gestr. 15 Esslöffel, gestrichen
UNMATCHED   18 EL, gestr. 18 Esslöffel, gestrichen
UNMATCHED   2 EL, gestr. 2 Esslöffel, gestrichen
UNMATCHED

In [18]:
len(parsed1), len(amounts)

(2782, 2782)

In [21]:
parsed1[100]

('0,33 Tasse', '0,33', 'Tasse')

In [10]:
re_num = re.compile(r"([\d½¼¾⅛]+)(?: ([½¼¾⅛]))?(?:,(\d+))?")

In [14]:
def parse_num(amount, unit):
    if amount is None:
        return (None, None)
    fs = {"½": 1/2, "¼": 1/4, "¾": 3/4, "⅛": 1/8}
    whole, addition, fraction = re_num.match(amount).groups()
    whole = fs[whole] if whole in fs else int(whole)
    addition = fs[addition] if addition is not None else 0
    fraction = float("0." + fraction) if fraction is not None else 0
    total_amount = whole + addition + fraction
    if unit == "kg":
        unit = "g"
        total_amount *= 1000
    if unit == "mg":
        unit = "g"
        total_amount /= 1000
    if unit and (unit.lower() == "l" or unit.lower() == "liter"):
        unit = "ml"
        total_amount *= 1000
    if unit == "cl":
        unit = "ml"
        total_amount *= 10
    return total_amount, unit

parsed_amounts = {orig: parse_num(n, e) for orig, n, e in parsed1}

In [15]:
parsed_amounts

{'': (1, None),
 '300 dl': (300, 'dl'),
 '500 Bund': (500, 'Bund'),
 '5 cm': (5, 'cm'),
 '177 ml': (177, 'ml'),
 '0,15 kg': (150.0, 'g'),
 '13 ½ g': (13.5, 'g'),
 '¾ m.-großer': (0.75, 'mittelgroßer'),
 '1 ½ kg': (1500.0, 'g'),
 '2 ½ gr. Gläser': (2.5, 'gr. Gläser'),
 '4 Kugeln': (4, 'Kugeln'),
 '0,2 Flasche': (0.2, 'Flasche'),
 '2 ½ Pck.': (2.5, 'Packung'),
 '¼ Kopf': (0.25, 'Kopf'),
 '5 Würfel': (5, 'Würfel'),
 '10 dicke': (10, 'dicke'),
 '800': (800, None),
 '10 ½ g': (10.5, 'g'),
 '3 ½ kg': (3500.0, 'g'),
 '24 Zehe/n': (24, 'Zehe'),
 '3 ½ cm': (3.5, 'cm'),
 '1750 ml': (1750, 'ml'),
 '14 m.-große': (14, 'mittelgroße'),
 'Paket': (1, 'Paket'),
 '6 Platte/n': (6, 'Platte'),
 '0,95 kg': (950.0, 'g'),
 '½ Liter': (500.0, 'ml'),
 '454 g': (454, 'g'),
 '4 Paket': (4, 'Paket'),
 '436 g': (436, 'g'),
 '400 m.-große': (400, 'mittelgroße'),
 '0,3 Ring/e': (0.3, 'Ring'),
 '0,3 g': (0.3, 'g'),
 '0,33 TL, gehäuft': (0.33, 'Teelöffel, gehäuft'),
 '135 ml': (135, 'ml'),
 '5 Kugel/n': (5, 'Kugel'),

In [16]:
with open("data/recipes/parsed_amounts.json", "w") as f:
    json.dump(parsed_amounts, f)